In [1]:
# Load necessary libraries
library(readr)
library(dplyr)
library(tidytext)
library(tm)
library(topicmodels)
library(ggplot2)
library(parallel) 
library(slam) 
library(Rmpfr)  
library(reshape2) 
library(scales) 
library(grid)
library(ldatuning)
library(textmineR)
library(SnowballC)
library(textstem)
library(textclean)
library(stringr)
library(qdapDictionaries)
library(lexicon)
library(here)
library(lubridate)
library(text2vec)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: NLP


Attaching package: ‘ggplot2’


The following object is masked from ‘package:NLP’:

    annotate


Loading required package: gmp


Attaching package: ‘gmp’


The following objects are masked from ‘package:base’:

    %*%, apply, crossprod, matrix, tcrossprod


C code of R package 'Rmpfr': GMP using 64 bits per limb



Attaching package: ‘Rmpfr’


The following object is masked from ‘package:gmp’:

    outer


The following objects are masked from ‘package:stats’:

    dbinom, dgamma, dnbinom, dnorm, dpois, dt, pnorm


The following objects are masked from ‘package:base’:

    cbind, pmax, pmin, rbind



Attaching package: ‘scales’


The following object is masked from ‘package:readr’:

    col_factor


Loading required package: Matrix


Attaching package: ‘t

In [2]:
i_am("here.txt")
here()

here() starts at /home/ec2-user/SageMaker/esa_analysis



[1] "/home/ec2-user/SageMaker/esa_analysis"

In [3]:
clean_corpus <- read.csv(
    here("data", "processed", "clean_text_corpus.csv"))

str(clean_corpus)

'data.frame':	5239 obs. of  6 variables:
 $ X.1  : int  1 2 3 4 5 6 7 8 9 10 ...
 $ X    : int  0 1 2 3 4 5 6 7 8 9 ...
 $ GOID : num  4.22e+08 4.09e+08 4.29e+08 4.19e+08 2.07e+09 ...
 $ Title: chr  "Developers Urge Delay of Gnatcatcher Ruling; Birds: Species isn't in danger, they claim. Federal decision this "| __truncated__ "TODAY IN CONGRESS" "Plan Offered to Aid Northwest Salmon and Trout" "SAVING WILDLIFE" ...
 $ Date : chr  "2000-09-26" "1999-07-20" "1994-03-27" "1999-07-05" ...
 $ Text : chr  "arm study developer demand federal official postpone decision due week provide acre critical habitat tiny gnatc"| __truncated__ "senate meet a.m. committee arm service a.m. u.s. policy military operation kosovo defense sec william cohen joi"| __truncated__ "ap clinton administration propose protection zone river stream threaten fish species federal land eastern orego"| __truncated__ "tribune june news item recovery propose removal bald eagle endanger species list mention eagle extinction m

In [4]:
# Create DTM using TF-IDF
tokens <- word_tokenizer(clean_corpus$Text)
it <- itoken(tokens, ids = clean_corpus$GOID, progressbar = FALSE)
vocab <- create_vocabulary(it)
vectorizer <- vocab_vectorizer(vocab)
dtm <- create_dtm(it, vectorizer)

# Use TF-IDF
tfidf <- TfIdf$new()
dtm_tfidf <- fit_transform(dtm, tfidf)


In [5]:
dim(dtm_tfidf)

[1]  5239 44941

In [6]:
# Compute cosine similarity
cosine_sim <- sim2(dtm_tfidf, method = "cosine", norm = "l2")

In [7]:
str(cosine_sim)
dim(cosine_sim)

Formal class 'dsCMatrix' [package "Matrix"] with 7 slots
  ..@ i       : int [1:13726180] 0 0 1 0 1 2 0 1 2 3 ...
  ..@ p       : int [1:5240] 0 1 3 6 10 15 21 28 36 45 ...
  ..@ Dim     : int [1:2] 5239 5239
  ..@ Dimnames:List of 2
  .. ..$ : chr [1:5239] "421721236" "408507519" "429461998" "418798522" ...
  .. ..$ : chr [1:5239] "421721236" "408507519" "429461998" "418798522" ...
  ..@ x       : num [1:13726180] 1 0.013 1 0.0361 0.0262 ...
  ..@ uplo    : chr "U"
  ..@ factors : list()


[1] 5239 5239

In [8]:
saveRDS(
    cosine_sim,
    here("data", "processed", "whole_cosine_sim.rds")
)

In [9]:
# Identify syndicated articles based on threshold
similarity_threshold <- 0.95

# Identify syndicated articles based on the threshold
pairs <- expand.grid(Article1 = clean_corpus$GOID, Article2 = clean_corpus$GOID) %>%
  filter(Article1 < Article2) %>%  # Ensure unique pairs and exclude self-comparisons
  mutate(
    Similarity = cosine_sim[cbind(
      match(Article1, clean_corpus$GOID),
      match(Article2, clean_corpus$GOID)
    )]
  ) %>%
  filter(Similarity >= similarity_threshold)


In [10]:
str(pairs)
head(pairs, n = 10)

'data.frame':	233 obs. of  3 variables:
 $ Article1  : num  2.10e+09 4.21e+08 4.30e+08 7.59e+08 4.21e+08 ...
 $ Article2  : num  2.10e+09 4.21e+08 4.30e+08 7.64e+08 4.21e+08 ...
 $ Similarity: num  0.972 0.992 1 0.983 0.985 ...
 - attr(*, "out.attrs")=List of 2
  ..$ dim     : Named int [1:2] 5239 5239
  .. ..- attr(*, "names")= chr [1:2] "Article1" "Article2"
  ..$ dimnames:List of 2
  .. ..$ Article1: chr [1:5239] "Article1= 421721236" "Article1= 408507519" "Article1= 429461998" "Article1= 418798522" ...
  .. ..$ Article2: chr [1:5239] "Article2= 421721236" "Article2= 408507519" "Article2= 429461998" "Article2= 418798522" ...


,Article1,Article2,Similarity
,<dbl>,<dbl>,<dbl>
1,2100798323,2100798380,0.9720786
2,421314883,421347484,0.9923435
3,430332811,430333805,1.0000000
4,758820654,763701373,0.9828463
5,421472896,421479391,0.9852781
6,2630547628,2630564268,0.9547288
7,420415753,422099712,0.9509504
8,293195609,293262146,0.9933331
9,421470906,421476020,0.9616118


In [11]:
# Initialize group tracking
pairs2 <- pairs %>%
  mutate(Group = NA_integer_)

head(pairs2)

,Article1,Article2,Similarity,Group
,<dbl>,<dbl>,<dbl>,<int>
1,2100798323,2100798380,0.9720786,NA
2,421314883,421347484,0.9923435,NA
3,430332811,430333805,1.0000000,NA
4,758820654,763701373,0.9828463,NA
5,421472896,421479391,0.9852781,NA
6,2630547628,2630564268,0.9547288,NA


In [12]:
group_id <- 1

# Assign groups iteratively
while (any(is.na(pairs2$Group))) {
  # Get the first ungrouped pair
  ungrouped <- pairs2 %>% filter(is.na(Group)) %>% slice(1)
  current_group <- c(ungrouped$Article1, ungrouped$Article2)
  
  # Expand the group to include all connected articles
  repeat {
    new_members <- pairs2 %>%
      filter(is.na(Group) & (Article1 %in% current_group | Article2 %in% current_group)) %>%
      select(Article1, Article2) %>%
      unlist() %>%
      unique()
    
    if (all(new_members %in% current_group)) break  # No new members
    current_group <- unique(c(current_group, new_members))
  }
  
  # Assign group ID to all pairs in this group
  pairs2 <- pairs2 %>%
    mutate(Group = ifelse(Article1 %in% current_group | Article2 %in% current_group, group_id, Group))
  
  group_id <- group_id + 1  # Increment group ID
}

In [13]:
str(pairs2)
head(pairs2)
range(pairs2$Group)

'data.frame':	233 obs. of  4 variables:
 $ Article1  : num  2.10e+09 4.21e+08 4.30e+08 7.59e+08 4.21e+08 ...
 $ Article2  : num  2.10e+09 4.21e+08 4.30e+08 7.64e+08 4.21e+08 ...
 $ Similarity: num  0.972 0.992 1 0.983 0.985 ...
 $ Group     : num  1 2 3 4 5 6 7 8 9 10 ...
 - attr(*, "out.attrs")=List of 2
  ..$ dim     : Named int [1:2] 5239 5239
  .. ..- attr(*, "names")= chr [1:2] "Article1" "Article2"
  ..$ dimnames:List of 2
  .. ..$ Article1: chr [1:5239] "Article1= 421721236" "Article1= 408507519" "Article1= 429461998" "Article1= 418798522" ...
  .. ..$ Article2: chr [1:5239] "Article2= 421721236" "Article2= 408507519" "Article2= 429461998" "Article2= 418798522" ...


,Article1,Article2,Similarity,Group
,<dbl>,<dbl>,<dbl>,<dbl>
1,2100798323,2100798380,0.9720786,1
2,421314883,421347484,0.9923435,2
3,430332811,430333805,1.0000000,3
4,758820654,763701373,0.9828463,4
5,421472896,421479391,0.9852781,5
6,2630547628,2630564268,0.9547288,6


[1]   1 166

In [14]:
group1 <- pairs2 %>% 
    select("GOID" = Article1, Group) %>% 
    distinct()

str(group1)
head(group1)

'data.frame':	184 obs. of  2 variables:
 $ GOID : num  2.10e+09 4.21e+08 4.30e+08 7.59e+08 4.21e+08 ...
 $ Group: num  1 2 3 4 5 6 7 8 9 10 ...
 - attr(*, "out.attrs")=List of 2
  ..$ dim     : Named int [1:2] 5239 5239
  .. ..- attr(*, "names")= chr [1:2] "Article1" "Article2"
  ..$ dimnames:List of 2
  .. ..$ Article1: chr [1:5239] "Article1= 421721236" "Article1= 408507519" "Article1= 429461998" "Article1= 418798522" ...
  .. ..$ Article2: chr [1:5239] "Article2= 421721236" "Article2= 408507519" "Article2= 429461998" "Article2= 418798522" ...


,GOID,Group
,<dbl>,<dbl>
1,2100798323,1
2,421314883,2
3,430332811,3
4,758820654,4
5,421472896,5
6,2630547628,6


In [15]:
group2 <- pairs2 %>% 
    select("GOID" = Article2, Group) %>% 
    distinct()

str(group2)
head(group2)

'data.frame':	185 obs. of  2 variables:
 $ GOID : num  2.10e+09 4.21e+08 4.30e+08 7.64e+08 4.21e+08 ...
 $ Group: num  1 2 3 4 5 6 7 8 9 10 ...
 - attr(*, "out.attrs")=List of 2
  ..$ dim     : Named int [1:2] 5239 5239
  .. ..- attr(*, "names")= chr [1:2] "Article1" "Article2"
  ..$ dimnames:List of 2
  .. ..$ Article1: chr [1:5239] "Article1= 421721236" "Article1= 408507519" "Article1= 429461998" "Article1= 418798522" ...
  .. ..$ Article2: chr [1:5239] "Article2= 421721236" "Article2= 408507519" "Article2= 429461998" "Article2= 418798522" ...


,GOID,Group
,<dbl>,<dbl>
1,2100798380,1
2,421347484,2
3,430333805,3
4,763701373,4
5,421479391,5
6,2630564268,6


In [16]:
syndicated_groups <- rbind(group2, group1) %>% distinct()

str(syndicated_groups)
head(syndicated_groups)

'data.frame':	351 obs. of  2 variables:
 $ GOID : num  2.10e+09 4.21e+08 4.30e+08 7.64e+08 4.21e+08 ...
 $ Group: num  1 2 3 4 5 6 7 8 9 10 ...
 - attr(*, "out.attrs")=List of 2
  ..$ dim     : Named int [1:2] 5239 5239
  .. ..- attr(*, "names")= chr [1:2] "Article1" "Article2"
  ..$ dimnames:List of 2
  .. ..$ Article1: chr [1:5239] "Article1= 421721236" "Article1= 408507519" "Article1= 429461998" "Article1= 418798522" ...
  .. ..$ Article2: chr [1:5239] "Article2= 421721236" "Article2= 408507519" "Article2= 429461998" "Article2= 418798522" ...


,GOID,Group
,<dbl>,<dbl>
1,2100798380,1
2,421347484,2
3,430333805,3
4,763701373,4
5,421479391,5
6,2630564268,6


In [17]:
count <- syndicated_groups %>% 
    count(Group) %>% 
    arrange(desc(n))

str(count)
head(count)

'data.frame':	166 obs. of  2 variables:
 $ Group: num  12 30 18 19 24 27 74 83 100 1 ...
 $ n    : int  12 4 3 3 3 3 3 3 3 2 ...
 - attr(*, "out.attrs")=List of 2
  ..$ dim     : Named int [1:2] 5239 5239
  .. ..- attr(*, "names")= chr [1:2] "Article1" "Article2"
  ..$ dimnames:List of 2
  .. ..$ Article1: chr [1:5239] "Article1= 421721236" "Article1= 408507519" "Article1= 429461998" "Article1= 418798522" ...
  .. ..$ Article2: chr [1:5239] "Article2= 421721236" "Article2= 408507519" "Article2= 429461998" "Article2= 418798522" ...


,Group,n
,<dbl>,<int>
1,12,12
2,30,4
3,18,3
4,19,3
5,24,3
6,27,3


In [18]:
group_check <- syndicated_groups %>% 
    filter(Group == "12")

group_check

GOID,Group
<dbl>,<dbl>
409464531,12
409520608,12
409870361,12
409874944,12
409345592,12
409301278,12
409592729,12
409559555,12
409298559,12


In [19]:
# Get syndicated and unique articles
syndicated_ids <- syndicated_groups$GOID

# Syndicated articles
syndicated_articles <- clean_corpus %>%
  filter(GOID %in% syndicated_ids) %>% 
  inner_join(syndicated_groups, by = "GOID")

In [20]:
str(syndicated_ids)
str(syndicated_articles)

 num [1:351] 2.10e+09 4.21e+08 4.30e+08 7.64e+08 4.21e+08 ...
'data.frame':	351 obs. of  7 variables:
 $ X.1  : int  8 21 40 42 48 57 60 87 108 115 ...
 $ X    : int  7 20 39 41 47 56 59 86 107 114 ...
 $ GOID : num  2.10e+09 4.21e+08 4.21e+08 1.81e+09 1.17e+09 ...
 $ Title: chr  "Legal Notice 2 -- No Title" "EDUCATION: SMART RESOURCES FOR STUDENTS AND PARENTS; Where Learning and the Internet Meet: LAUNCH POINT; Endangered Species" "VENTURA COUNTY NEWS; Groups Join to Push Steelhead Safeguards" "Island foxes taken off endangered species list" ...
 $ Date : chr  "1998-01-05" "1998-11-02" "2000-02-15" "2016-08-13" ...
 $ Text : chr  "legal notice service bid want request proposal california tern colony monitor study harbor department city los "| __truncated__ "high rate plant animal extinction alarm unite state species endanger threaten due habitat loss pollution introd"| __truncated__ "unusual coalition interest announce monday band press great protection steelhead trout fast disappear 

In [21]:
write.csv(
    syndicated_articles,
    here("data", "processed", "syndicated_articles.csv")
)

In [22]:
# Keep only the earliest article from each group, resolving ties by Article_ID
earliest_syndicated <- syndicated_articles %>%
  group_by(Group) %>%
  slice_min(order_by = Date, with_ties = TRUE) %>% # Keep ties for the same date
  slice_min(order_by = GOID, n = 1) %>%      # Pick one article (arbitrarily by ID)
  ungroup()

In [23]:
str(earliest_syndicated)

tibble [166 × 7] (S3: tbl_df/tbl/data.frame)
 $ X.1  : int [1:166] 659 135 797 5127 1030 3398 4336 251 3401 2113 ...
 $ X    : int [1:166] 658 134 796 5128 1029 3399 4337 250 3402 2113 ...
 $ GOID : num [1:166] 2.10e+09 4.21e+08 4.30e+08 7.59e+08 4.21e+08 ...
 $ Title: chr [1:166] "Legal Notice 1 -- No Title" "EDUCATION / An Exploration of Ideas, Issues and Trends in Education; Where Learning and the Internet Meet: LAUN"| __truncated__ "Moderates Soften G.O.P. Agenda on Environment" "A pack of lawmakers goes hunting; New legislation would keep the resurgent gray wolf off the federal endangered species list." ...
 $ Date : chr [1:166] "1998-01-05" "1998-11-02" "1995-10-24" "2010-10-18" ...
 $ Text : chr [1:166] "legal notice service bid want request proposal california tern colony monitor study harbor department city los "| __truncated__ "high rate plant animal extinction alarm unite state species endanger threaten due habitat loss pollution introd"| __truncated__ "tacit support party l

In [24]:
earliest_syndicated2 <- earliest_syndicated %>% 
    select(-Group)

str(earliest_syndicated2)

tibble [166 × 6] (S3: tbl_df/tbl/data.frame)
 $ X.1  : int [1:166] 659 135 797 5127 1030 3398 4336 251 3401 2113 ...
 $ X    : int [1:166] 658 134 796 5128 1029 3399 4337 250 3402 2113 ...
 $ GOID : num [1:166] 2.10e+09 4.21e+08 4.30e+08 7.59e+08 4.21e+08 ...
 $ Title: chr [1:166] "Legal Notice 1 -- No Title" "EDUCATION / An Exploration of Ideas, Issues and Trends in Education; Where Learning and the Internet Meet: LAUN"| __truncated__ "Moderates Soften G.O.P. Agenda on Environment" "A pack of lawmakers goes hunting; New legislation would keep the resurgent gray wolf off the federal endangered species list." ...
 $ Date : chr [1:166] "1998-01-05" "1998-11-02" "1995-10-24" "2010-10-18" ...
 $ Text : chr [1:166] "legal notice service bid want request proposal california tern colony monitor study harbor department city los "| __truncated__ "high rate plant animal extinction alarm unite state species endanger threaten due habitat loss pollution introd"| __truncated__ "tacit support party l

In [25]:
# Unique articles
unique_articles <- clean_corpus %>%
  filter(!GOID %in% syndicated_ids)

str(unique_articles)

'data.frame':	4888 obs. of  6 variables:
 $ X.1  : int  1 2 3 4 5 6 7 9 10 11 ...
 $ X    : int  0 1 2 3 4 5 6 8 9 10 ...
 $ GOID : num  4.22e+08 4.09e+08 4.29e+08 4.19e+08 2.07e+09 ...
 $ Title: chr  "Developers Urge Delay of Gnatcatcher Ruling; Birds: Species isn't in danger, they claim. Federal decision this "| __truncated__ "TODAY IN CONGRESS" "Plan Offered to Aid Northwest Salmon and Trout" "SAVING WILDLIFE" ...
 $ Date : chr  "2000-09-26" "1999-07-20" "1994-03-27" "1999-07-05" ...
 $ Text : chr  "arm study developer demand federal official postpone decision due week provide acre critical habitat tiny gnatc"| __truncated__ "senate meet a.m. committee arm service a.m. u.s. policy military operation kosovo defense sec william cohen joi"| __truncated__ "ap clinton administration propose protection zone river stream threaten fish species federal land eastern orego"| __truncated__ "tribune june news item recovery propose removal bald eagle endanger species list mention eagle extinction

In [26]:
# Combine unique articles with earliest syndicated articles
final_corpus <- bind_rows(unique_articles, earliest_syndicated2) %>%
  arrange(Date) %>% 
  select(GOID, Title, Date, Text)

str(final_corpus)
head(final_corpus)

'data.frame':	5054 obs. of  4 variables:
 $ GOID : num  4.24e+08 4.24e+08 4.24e+08 4.24e+08 4.24e+08 ...
 $ Title: chr  "BY TOM FERRELL AND MARGOT SLADE; IDEAS AND TRENDS; Pope's Clear Call To Serve Men, Not Their Systems" "In Legal Limbo" "THE CASE FOR THE BITTERING AND BRONX RIVER" "MORE SNAIL DARTERS FOUND IN TENNESSEE" ...
 $ Date : chr  "1980-07-13" "1980-09-07" "1980-10-19" "1980-11-08" ...
 $ Text : chr  "universal bishop roman catholic church pope john paul ii call fulfil preach office bishop end week tour brazil "| __truncated__ "case donald lang murder question legally mr lang unable speak hear read write find guilty chicago kill prostitu"| __truncated__ "bronx river survive ravage time assault civilization year hot dry summer record threaten danger channelization "| __truncated__ "believe population snail darter inch fish demise lead supreme court halt construction tellico dam tennessee yea"| __truncated__ ...


GOID     
1 423968056
2 423987935
3 423991776
4 424019341
5 424018521
6 424019099
  Title                                                                                               
1 BY TOM FERRELL AND MARGOT SLADE; IDEAS AND TRENDS; Pope's Clear Call To Serve Men, Not Their Systems
2 In Legal Limbo                                                                                      
3 THE CASE FOR THE BITTERING AND BRONX RIVER                                                          
4 MORE SNAIL DARTERS FOUND IN TENNESSEE                                                               
5 HUNTING THE ILLEGAL HUNTERS                                                                         
6 OIL DRILL STUDY MUDDIES WATERS FOR BOWHEAD WHALES OFF ALASKA                                        
  Date      
1 1980-07-13
2 1980-09-07
3 1980-10-19
4 1980-11-08
5 1980-11-09
6 1980-11-13
  Text                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [27]:
write.csv(
    final_corpus,
    here("data", "processed", "clean_original_docs_corpus.csv")
)